In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Генерация случайной выборки точек на координатной плоскости 100x100: 70% - участвуют, из них 95% - маркируются по сумме координат, а оставльные 5% - рандомно
xs = list(np.arange(0, 100)) * 100
ys = [item for sublist in list(map(lambda x: [x] * 100, np.arange(0, 100))) for item in sublist]
points = list(zip(xs, ys))
np.random.shuffle(points)
involved_points = points[:int(len(points) * 0.7)]
involved_points_95 = involved_points[:int(len(involved_points) * 0.95)]
involved_points_5 = involved_points[-int(len(involved_points) * 0.05):]

In [3]:
def label_points(fst, snd):
  if fst + snd < 30:
    return 'A'
  elif fst + snd < 65:
    return 'B'
  else:
    return 'C'
  
def random_label_points():
  rand = np.random.random()
  if rand < 1 / 3:
    return 'A'
  elif rand < 2 / 3:
    return 'B'
  else:
    return 'C'

# Маркировка данных
labeled_involved_points_95 = [(first, second, label_points(first, second)) for (first, second) in involved_points_95]
labeled_involved_points_5 = [(first, second, random_label_points()) for (first, second) in involved_points_5]
all_points = labeled_involved_points_95 + labeled_involved_points_5
knn_data = pd.DataFrame(all_points, columns=['x', 'y', 'attr'])
knn_data.head()

,x,y,attr
0,86,12,C
1,78,69,C
2,0,67,C
3,72,44,C
4,25,18,B


In [4]:
# Создание подмножеств классифицированных и некласифицированных данных
knn_classified_data = knn_data[:int(len(knn_data) * 0.9)]
knn_unclassified_data = knn_data[-int(len(knn_data) * 0.1):]

In [5]:
# Создание таблицы distanced - таблицы расстояний между классифицированными и неклассифицированными точками
# Внимание! Запрос выполняется за ~13 минут (возможно, стоит оптимизировать)
distanced_arr = []
for fst_index, class_row in knn_classified_data.iterrows():
  for snd_index, unclass_row in knn_unclassified_data.iterrows():
    distanced_arr.append((unclass_row['x'], unclass_row['y'], class_row['x'], class_row['y'], class_row['attr'], np.sqrt((unclass_row['x'] - class_row['x']) ** 2 + (unclass_row['y'] - class_row['y']) ** 2)))
distanced = pd.DataFrame(distanced_arr, columns=['x', 'y', 'x2', 'y2', 'attr', 'distance'])
distanced.head()

KeyboardInterrupt: 

In [6]:
# Добавление к полученной таблице столбца distance_rn - порядкового номера числа расстояния (по возрастанию) для текущей пары x, y (???)
distanced_row_numbered

In [7]:
# Выбрать все записи из полученной таблицы, distance_rn которых не превышает 5
top_n_selected = distanced_row_numbered[distanced_row_numbered["distance_rn"] <= 5]
top_n_selected.head()

NameError: name 'distanced' is not defined

In [8]:
# Подсчет частоты встречаемости у соседей (поле attr_cnt)
top_n_attr_counted

In [9]:
# Добавление к полученной таблице столбца attr_rn - порядкового номера числа количества упоминаний attr у соседей для этой пары x, y (???)
top_n_attr_counted_and_numbered

In [10]:
# Выбор самого часто встречаемого атрибута у соседей
classified_unclassified_data = top_n_attr_counted_and_numbered[top_n_attr_counted_and_numbered["attr_rn"] == 1][["x", "y", "attr"]]
classified_unclassified_data.head()

In [ ]:
# Подчсет точности классификации
diff_df = pd.merge(knn_data, classified_unclassified_data, how='outer', indicator='Exist')
diff_df = diff_df.loc[diff_df['Exist'] != 'both']
accuracy = len(diff_df) * 100 / len(knn_unclassified_data)
print(f"Точность классификации: {accuracy}")